In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Loading th data
data = pd.read_sas('/Users/maverick.shivaya/Downloads/HeinzHunts.sas7bdat', format='sas7bdat')

# Display the column names
print(data.columns)


Index(['HEINZ', 'HUNTS', 'PRICEHEINZ', 'PRICEHUNTS', 'FeatHeinz', 'FeatHunts',
       'DisplHeinz', 'DisplHunts'],
      dtype='object')


## Q1. A, B

In [3]:
# a. Create LogPriceRatio variable
data['LogPriceRatio'] = np.log(data['PRICEHEINZ'] / data['PRICEHUNTS'])

# b. Split the data into training and test samples
# Set a seed for reproducibility
train_data, test_data = train_test_split(data, test_size=0.25, random_state=123)

# Display the first few rows of the modified dataset
print(data.head())

   HEINZ  HUNTS  PRICEHEINZ  PRICEHUNTS  FeatHeinz  FeatHunts  DisplHeinz  \
0    1.0    0.0       0.052       0.034        0.0        0.0         0.0   
1    1.0    0.0       0.052       0.044        0.0        0.0         0.0   
2    1.0    0.0       0.046       0.048        1.0        0.0         0.0   
3    1.0    0.0       0.052       0.034        0.0        0.0         0.0   
4    1.0    0.0       0.046       0.048        1.0        0.0         0.0   

   DisplHunts  LogPriceRatio  
0         0.0       0.424883  
1         0.0       0.167054  
2         0.0      -0.042560  
3         0.0       0.424883  
4         0.0      -0.042560  


## C

In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm


# Rename columns to remove any potential inconsistencies with capitalization
data.columns = data.columns.str.upper()

# Create LogPriceRatio variable
data['LOGPRICERATIO'] = np.log(data['PRICEHEINZ'] / data['PRICEHUNTS'])

# Define the dependent variable (probability of purchasing Heinz)
data['HEINZ'] = data['HEINZ'].astype(int)  # Ensure 'HEINZ' is in integer format (1 for Heinz, 0 for Hunts)

# Add interaction terms for displays and features
data['DISPLHEINZ_FEATHEINZ'] = data['DISPLHEINZ'] * data['FEATHEINZ']
data['DISPLHUNTS_FEATHUNTS'] = data['DISPLHUNTS'] * data['FEATHUNTS']

# Define the independent variables
X = data[['LOGPRICERATIO', 'DISPLHEINZ', 'FEATHEINZ', 'DISPLHUNTS', 'FEATHUNTS', 'DISPLHEINZ_FEATHEINZ']]

# Add a constant for the intercept
X = sm.add_constant(X)

# Fit the logit model
y = data['HEINZ']
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Print the model summary
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.241449
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                  HEINZ   No. Observations:                 2798
Model:                          Logit   Df Residuals:                     2791
Method:                           MLE   Df Model:                            6
Date:                Mon, 28 Oct 2024   Pseudo R-squ.:                  0.3020
Time:                        23:01:04   Log-Likelihood:                -675.57
converged:                       True   LL-Null:                       -967.92
Covariance Type:            nonrobust   LLR p-value:                4.677e-123
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    3.2801      0.142     23.093      0.000       3.002       3.559

## Q2

In [5]:
import numpy as np

# Coefficients from the model output
const = 3.2801
coef_LogPriceRatio = -6.1109
coef_DisplHeinz = 0.5459
coef_FeatHeinz = 0.5760
coef_DisplHunts = -0.6685
coef_FeatHunts = -1.0935
coef_DisplHeinz_FeatHeinz = -0.6821

# Define the logit probability function
def logit_probability(logpriceratio, displheinz, featheinz, displhunts, feathunts):
    # Calculate the linear combination (log-odds)
    log_odds = (const + 
                coef_LogPriceRatio * logpriceratio + 
                coef_DisplHeinz * displheinz + 
                coef_FeatHeinz * featheinz + 
                coef_DisplHunts * displhunts + 
                coef_FeatHunts * feathunts + 
                coef_DisplHeinz_FeatHeinz * (displheinz * featheinz))
    
    # Convert log-odds to probability
    probability = np.exp(log_odds) / (1 + np.exp(log_odds))
    return probability

# Scenario 1: LogPriceRatio = 0.5
prob_1 = logit_probability(logpriceratio=0.5, displheinz=0, featheinz=0, displhunts=1, feathunts=1)

# Scenario 2: LogPriceRatio = 0.55
prob_2 = logit_probability(logpriceratio=0.55, displheinz=0, featheinz=0, displhunts=1, feathunts=1)

# Calculate the change in probability
change_in_probability = prob_2 - prob_1

# Display the results
print(f"Probability of purchasing Heinz at LogPriceRatio 0.5: {prob_1:.4f}")
print(f"Probability of purchasing Heinz at LogPriceRatio 0.55: {prob_2:.4f}")
print(f"Change in probability: {change_in_probability:.4f}")


Probability of purchasing Heinz at LogPriceRatio 0.5: 0.1769
Probability of purchasing Heinz at LogPriceRatio 0.55: 0.1367
Change in probability: -0.0402


## Q3

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm

# Define the independent variables for prediction
X = data[['LOGPRICERATIO', 'DISPLHEINZ', 'FEATHEINZ', 'DISPLHUNTS', 'FEATHUNTS', 'DISPLHEINZ_FEATHEINZ']]
X = sm.add_constant(X)

# Generate predicted probabilities from the model
data['predicted_prob_heinz'] = result.predict(X)

# Costs for misclassifications
cost_false_positive = 1.5  # Cost of sending coupon to a Heinz buyer
cost_false_negative = 0.25  # Cost of not sending coupon to a Hunts buyer

# Define a range of thresholds to test
thresholds = np.arange(0, 1.01, 0.01)
costs = []

# Iterate over each threshold
for threshold in thresholds:
    # Predict based on the threshold
    data['predicted_class'] = np.where(data['predicted_prob_heinz'] >= threshold, 1, 0)  # 1 = Heinz, 0 = Hunts
    
    # Calculate confusion matrix components
    tn, fp, fn, tp = confusion_matrix(data['HEINZ'], data['predicted_class']).ravel()
    
    # Calculate misclassification costs
    total_cost = (fp * cost_false_positive) + (fn * cost_false_negative)
    costs.append(total_cost)

# Find the optimal threshold with the minimum cost
optimal_threshold = thresholds[np.argmin(costs)]
min_cost = min(costs)

print(f"Optimal Threshold: {optimal_threshold:.2f}")
print(f"Minimum Cost at Optimal Threshold: ${min_cost:.2f}")


Optimal Threshold: 0.89
Minimum Cost at Optimal Threshold: $222.00
